In [2]:
import casadi as ca
import sympy as sp
import numpy as np
from lib.utils import sympy2casadi

In [3]:
num_poles = 2

pole_as = [0.1 for i in range(num_poles)]
pole_ls = [0.2 for i in range(num_poles)]
pole_ms = [0.05 for i in range(num_poles)]
pole_ds = [0.01 for i in range(num_poles)]
pole_Js = [0.1 for i in range(num_poles)]

m_c = 0.5
g = 9.81

In [6]:
t = sp.symbols("t")
s = sp.Function("s")(t) #type: ignore
d_s = sp.diff(s, t)
dd_s = sp.diff(d_s, t)
thetas = [sp.Function(f"theta{i+1}")(t) for i in range(num_poles)] #type: ignore
d_thetas = [sp.diff(theta,t) for theta in thetas]
dd_thetas = [sp.diff(d_theta,t) for d_theta in d_thetas]

pole_pc1s = []
pole_pc2s = []
for i, (theta, a) in enumerate(zip(thetas, pole_as)):
    prev_1 = 0
    prev_2 = 0
    for prev_l, prev_theta in list(zip(pole_ls, thetas))[:i]:
        prev_1 += -prev_l*sp.sin(prev_theta)
        prev_2 += prev_l*sp.cos(prev_theta)
    pole_pc1s.append(s-a*sp.sin(theta)+prev_1)
    pole_pc2s.append(a*sp.cos(theta)+prev_2)

T = 1/2*m_c*d_s**2
for m, pc1, pc2, J, d_theta in zip(pole_ms, pole_pc1s, pole_pc2s, pole_Js, d_thetas):
    d_pc1 = sp.diff(pc1,t)
    d_pc2 = sp.diff(pc2,t)
    T += 1/2*m*(d_pc1**2 + d_pc2**2) + 1/2*J*d_theta**2 #type: ignore
    
V = 0   
for m, pc2 in zip(pole_ms, pole_pc2s):
    V += g*m*pc2

R = 0
prev_w = 0
for d, d_theta in zip(pole_ds, d_thetas):
    R += 1/2*d*(d_theta-prev_w)**2 #type: ignore
    prev_w = d_theta

eqs = []
for theta, d_theta in zip(thetas, d_thetas):
    L = T-V
    lh = sp.diff(sp.diff(L, d_theta),t) - sp.diff(L, theta) + sp.diff(R, d_theta) #type: ignore
    rh = 0
    eqs.append(lh-rh)

sp_vars = [s, d_s] + [item for pair in zip(thetas, d_thetas) for item in pair] + [dd_s] + dd_thetas
sols = sp.solve(eqs, dd_thetas)
sp_sols = [sp.simplify(sols[dd_theta]) for dd_theta in dd_thetas]

In [7]:
for sol in sp_sols:
    print("Sol")
    display(sol)

Sol


(-0.00238101016965608*sin(theta1(t) - 2*theta2(t)) + 0.00975704473192398*sin(theta1(t) - theta2(t))*Derivative(theta2(t), t)**2 + 4.85425111040994e-5*sin(2*theta1(t) - 2*theta2(t))*Derivative(theta1(t), t)**2 - 1.43336812213296*sin(theta1(t)) + 0.000242712555520497*cos(theta1(t) - 2*theta2(t))*Derivative(s(t), (t, 2)) + 0.000970850222081988*cos(theta1(t) - theta2(t))*Derivative(theta1(t), t) - 0.000970850222081988*cos(theta1(t) - theta2(t))*Derivative(theta2(t), t) - 4.06575814682064e-20*cos(theta1(t) + 2*theta2(t))*Derivative(s(t), (t, 2)) - 0.146112958423339*cos(theta1(t))*Derivative(s(t), (t, 2)) + 0.19514089463848*Derivative(theta1(t), t) - 0.0975704473192398*Derivative(theta2(t), t))/(4.85425111040994e-5*cos(2*theta1(t) - 2*theta2(t)) - 1.0000485425111)

Sol


(-0.00995121477634038*sin(theta1(t) - theta2(t))*Derivative(theta1(t), t)**2 - 4.85425111040994e-5*sin(2*theta1(t) - 2*theta2(t))*Derivative(theta2(t), t)**2 + 0.00714303050896823*sin(2*theta1(t) - theta2(t)) - 0.480964054270527*sin(theta2(t)) - 0.00194170044416398*cos(theta1(t) - theta2(t))*Derivative(theta1(t), t) + 0.000970850222081989*cos(theta1(t) - theta2(t))*Derivative(theta2(t), t) + 0.000728137666561491*cos(2*theta1(t) - theta2(t))*Derivative(s(t), (t, 2)) - 0.0490279362151404*cos(theta2(t))*Derivative(s(t), (t, 2)) - 0.0995121477634038*Derivative(theta1(t), t) + 0.0995121477634038*Derivative(theta2(t), t))/(4.85425111040994e-5*cos(2*theta1(t) - 2*theta2(t)) - 1.0000485425111)